# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import socket, struct

In [10]:
datasetPath = "Dataset/Dataset-Applications-87Attributes.csv"
df = pd.read_csv(datasetPath)
#df = pd.read_csv(datasetPath, nrows=1000)
df.head()

,Flow.ID,Source.IP,Source.Port,Destination.IP,Destination.Port,Protocol,Timestamp,Flow.Duration,Total.Fwd.Packets,Total.Backward.Packets,...,Active.Std,Active.Max,Active.Min,Idle.Mean,Idle.Std,Idle.Max,Idle.Min,Label,L7Protocol,ProtocolName
0,172.19.1.46-10.200.7.7-52422-3128-6,172.19.1.46,52422,10.200.7.7,3128,6,26/04/201711:11:17,45523,22,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
1,172.19.1.46-10.200.7.7-52422-3128-6,10.200.7.7,3128,172.19.1.46,52422,6,26/04/201711:11:17,1,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY
2,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,1,3,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
3,10.200.7.217-50.31.185.39-38848-80-6,50.31.185.39,80,10.200.7.217,38848,6,26/04/201711:11:17,217,1,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,7,HTTP
4,192.168.72.43-10.200.7.7-55961-3128-6,192.168.72.43,55961,10.200.7.7,3128,6,26/04/201711:11:17,78068,5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,131,HTTP_PROXY


In [11]:
originalShape = df.shape

In [12]:
# Checking types of values
print(df.dtypes)

Flow.ID              object
Source.IP            object
Source.Port           int64
Destination.IP       object
Destination.Port      int64
                     ...   
Idle.Max            float64
Idle.Min            float64
Label                object
L7Protocol            int64
ProtocolName         object
Length: 87, dtype: object


### ip to int converter

In [13]:
def ip2int(addr):
    # convert ip to int
    return struct.unpack("!I", socket.inet_aton(addr))[0]

# Converting IP addresses into numbers
df['Source.IP'] = df['Source.IP'].apply(ip2int)
df['Destination.IP'] = df['Destination.IP'].apply(ip2int)

### Drop some of the unnecessary elements

In [14]:
#df.drop(['Source.IP', 'Destination.IP'], axis = 1, inplace = True)
#df.drop(['Source.Port', 'Destination.Port'], axis = 1, inplace = True)

# remove ['Flow.ID', 'Timestamp', 'Label'] columns
df.drop(['Flow.ID', 'Timestamp', 'Label'], axis = 1, inplace = True)

# it may necessary to remove columns with only single value
singleValueColumns = [col for col in df.columns if df[col].nunique() == 1]
df.drop(singleValueColumns, axis = 1, inplace = True)
df.shape

(3577296, 74)

In [15]:
# Checking types of values
print(df.dtypes)

Source.IP             int64
Source.Port           int64
Destination.IP        int64
Destination.Port      int64
Protocol              int64
                     ...   
Idle.Std            float64
Idle.Max            float64
Idle.Min            float64
L7Protocol            int64
ProtocolName         object
Length: 74, dtype: object


In [16]:
print("original shape:", originalShape, " -  new shape:", df.shape)
df.to_csv("Dataset/filtered-" + str(len(df.dtypes)) + "Attributes.csv", index = False, sep=',')

original shape: (3577296, 87)  -  new shape: (3577296, 74)


### 17 of the names contribute to %98.6

In [17]:
dfMost17 = df[df['ProtocolName'].map(df['ProtocolName'].value_counts()) > df['ProtocolName'].value_counts()[17]]

dfMost17['ProtocolName'].value_counts()

GOOGLE            959110
HTTP              683734
HTTP_PROXY        623210
SSL               404883
HTTP_CONNECT      317526
YOUTUBE           170781
AMAZON             86875
MICROSOFT          54710
GMAIL              40260
WINDOWS_UPDATE     34471
SKYPE              30657
FACEBOOK           29033
DROPBOX            25102
YAHOO              21268
TWITTER            18259
CLOUDFLARE         14737
MSN                14478
Name: ProtocolName, dtype: int64

In [18]:
dfMost17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3529094 entries, 0 to 3577295
Data columns (total 74 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Source.IP                    int64  
 1   Source.Port                  int64  
 2   Destination.IP               int64  
 3   Destination.Port             int64  
 4   Protocol                     int64  
 5   Flow.Duration                int64  
 6   Total.Fwd.Packets            int64  
 7   Total.Backward.Packets       int64  
 8   Total.Length.of.Fwd.Packets  int64  
 9   Total.Length.of.Bwd.Packets  float64
 10  Fwd.Packet.Length.Max        int64  
 11  Fwd.Packet.Length.Min        int64  
 12  Fwd.Packet.Length.Mean       float64
 13  Fwd.Packet.Length.Std        float64
 14  Bwd.Packet.Length.Max        int64  
 15  Bwd.Packet.Length.Min        int64  
 16  Bwd.Packet.Length.Mean       float64
 17  Bwd.Packet.Length.Std        float64
 18  Flow.Bytes.s                 float64
 19  

In [19]:
print("original shape:", originalShape, " -  new shape:", dfMost17.shape)
dfMost17.to_csv("Dataset/filtered-" + str(len(dfMost17.dtypes)) + "Attributes-top17_ProtocolName.csv", index = False, sep=',')

original shape: (3577296, 87)  -  new shape: (3529094, 74)


###  Balance dataset

In [20]:
# get minimum sample size
uniqueNames = dfMost17['ProtocolName'].unique()
minimumSampleSize = dfMost17['ProtocolName'].value_counts().min()

In [21]:
# balance dataset
balanced = pd.DataFrame()
for name in uniqueNames:
    g = dfMost17[dfMost17['ProtocolName']==name].sample(minimumSampleSize)
    balanced = pd.concat([balanced, g], axis=0)



In [22]:
balanced['ProtocolName'].value_counts()

YAHOO             14478
CLOUDFLARE        14478
AMAZON            14478
HTTP_CONNECT      14478
GOOGLE            14478
SKYPE             14478
DROPBOX           14478
MICROSOFT         14478
HTTP_PROXY        14478
FACEBOOK          14478
MSN               14478
YOUTUBE           14478
TWITTER           14478
WINDOWS_UPDATE    14478
HTTP              14478
GMAIL             14478
SSL               14478
Name: ProtocolName, dtype: int64

In [23]:
print(len(balanced.dtypes))

74


### Save balanced dataset

In [24]:
print("original shape:", originalShape, " -  new shape:", balanced.shape)
balanced.to_csv("Dataset/balanced-" + str(len(balanced.dtypes)) + "Attributes-top17_ProtocolName.csv", index = False, sep=',')

original shape: (3577296, 87)  -  new shape: (246126, 74)


### Remaining dataset may use for test 

In [ ]:
dfMost17 = dfMost17 - balanced

print("original shape:", originalShape, " -  new shape:", dfMost17.shape)
dfMost17 - balanced.to_csv("Dataset/Remaining-" + str(len(dfMost17)) + "Attributes-top17_ProtocolName.csv", index = False, sep=',')